Sam Odle <br>
December 2021  <br>
Project 3  <br>


In [ ]:
import torch

# Whether to train on a gpu
train_on_gpu = torch.cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

# Number of gpus
if train_on_gpu:
    gpu_count = torch.cuda.device_count()
    print(f'{gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False

CUDA_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(CUDA_DEVICE)
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__CUDA Device Name:',torch.cuda.get_device_name(0))
print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)

# Part 1: Generating Test Cases
###### In this section, we're going to design some datasets to convince ourselves that our model is giving us reasonable results. First, we'll generate some toy datasets from the Stanford dogs dataset where we know what the correct outcome should be. Complete the items below:

###### Generate a dataset of just three images, one for each class, and show your model correctly labels them. (display each image in your notebook, pass it to your model, and then print the prediction).	5 points

###### Generate three datasets of our inputs, where each has only two of the classes. What do you predict the performance should be for three binary classifiers trained on these three datasets? Re-train your model on these three datasets, and discuss your results.	5 points

###### Generate a dataset from your original dataset where 20% of the classes in one class are mis-labelled as the remaining two classes. How do you think your model performance will be impacted? Re-train your model on this test dataset, and discuss your results.

# Part 2: Biases in the modeling

###### Take a look at each of the images in all classes individually. What aspects of the image (such as backgrounds) might be influencing the decision-making of the model, besides the dogs themselves?	5 points
###### Calculate the "average image" across all pixels of each of your classes in your training dataset. Are your results consistent with the previous item?	5 points
###### Is the data biased in any way that could impact your results? Why or why not?
###### If you noted some potential biases in the modeling/dataset above, discuss how you could help mitigate these biases (you don't need to implement, just discuss). If you didn't note any biases in this dataset, discuss what biases there could have been, and how the dataset designers might have helped mitigate them.

Part 3: Model uncertainty and explainability
We discussed earlier this semester how deep learning models can be black boxes; it's hard to tell what the models really learned, that is, what are the using to make their decisions?

Overall, we got incredibly good performance earlier with just a couple hundred images per class. Let's investigate how much pre-training with ImageNet helped us achieve those results. First, re-train your model, but this time don't use a pre-trained version (you can set the flag to False). What happened? Include a discussion in your notebook.

Next, let's see what happens in two cases:
When we try to build a model on just the head of the dog (cropping just the head/face)
When we crop the dog out of the image, leaving just the background
Discuss two hypotheses for the two datasets above -- how do you think they should perform, and why. If you have time, you can manually build the two datasets above and re-train your model, but you're not required to.

Pixel importance with saliency maps
Finally, let's see how much each pixel contributes to the final decision of the model using saliency mapping. Although saliency mapping has its limitations and is not the be-all-end-all of this type of analysis, it will at least get us started this semester towards a discussion of model explainability.

Write a loop that goes through every image in your dataset and prints out the original image and a saliency map for each image. Use any resources you find online (other than asking someone to do the work for you!) to generate such a saliency map; you can probably do this in about 20 lines of code without the need to import any more libraries, but feel free to get fancy! If you get stuck, as us for help on Ed. Make sure you cite any resources you used!

Finally, include a discussion of the output of your saliency analysis for this dataset -- what do you think this means in terms of your model performance?


###### Discussion of how model should behave if trained just on heads of dogs
###### Discussion of how model should behave if trained just on backgrounds without dogs
###### Discussion of model performance without using pre-trained weights vs using pre-trained weights.
###### Correctly implementing saliency maps for all images.	5 points
###### Discussion of saliency mapping results.

In [ ]:
import os, shutil
import random

directory = '/Users/samodle/Documents/Repos/SimpleMachineLearningProjects/CNNs and Transfer Learning/Images/'

num_training_images_per_breed = 100
num_test_images_per_breed = 25
source_paths = ['n02109047-Great_Dane', 'n02094258-Norwich_terrier', 'n02094114-Norfolk_terrier']
destination_paths = ['train/', 'test/', 'holdout/']

dest_train = 0
dest_test = 1
dest_holdout = 2

for path in destination_paths:
    if not os.path.exists(directory + path):
      # Create a new directory because it does not exist
      os.makedirs(directory + path)
      print(f'Directory Created: {path}')
    else:
        print(f'Already Exists: {path}')

    if not os.path.exists(directory + path + 'great_dane/'):
        os.makedirs(directory + path + 'great_dane/')
        print(f'Directory Created: {path}')

    if not os.path.exists(directory + path + 'norwich_terrier/'):
        os.makedirs(directory + path + 'norwich_terrier/')

    if not os.path.exists(directory + path + 'norfolk_terrier/'):
        os.makedirs(directory + path + 'norfolk_terrier/')
        print(f'Directory Created: {path}')



In [ ]:
# split up the training, holdout, and test datasets
for path in source_paths:
    all_files = os.listdir(directory + path)

    #split out the training data
    train = random.sample(all_files, num_training_images_per_breed)

    if 'Great_Dane' in path:
        for f in train:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_train] + 'great_dane/')
    elif 'Norwich' in path:
         for f in train:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_train] + 'norwich_terrier/')
    elif 'Norfolk' in path:
        for f in train:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_train] + 'norfolk_terrier/')


    test_and_hold = [x for x in all_files if x not in train]
    # print(train)

    #split the test from the holdout
    test = random.sample(test_and_hold, num_test_images_per_breed)

    if 'Great_Dane' in path:
        for f in test:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_test] + 'great_dane/')
    elif 'Norwich' in path:
         for f in test:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_test] + 'norwich_terrier/')
    elif 'Norfolk' in path:
        for f in test:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_test] + 'norfolk_terrier/')


    holdout = [x for x in test_and_hold if x not in test]
    if 'Great_Dane' in path:
        for f in holdout:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_holdout] + 'great_dane/')
    elif 'Norwich' in path:
         for f in holdout:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_holdout] + 'norwich_terrier/')
    elif 'Norfolk' in path:
        for f in holdout:
            shutil.copy(directory + path + '/' + f, directory + destination_paths[dest_holdout] + 'norfolk_terrier/')

    print(test)
    print(holdout)

In [ ]:
import torchvision.transforms as transforms

train_transforms = transforms.Compose([
    transforms.Resize(size=[224,224]),
    transforms.RandomRotation(degrees=90),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])  # Imagenet standards
])

In [ ]:
test_transforms = transforms.Compose([
        transforms.Resize(size=[224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])

valid_transforms = transforms.Compose([
        transforms.Resize(size=[224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])

In [ ]:
import torch.utils.data as data
import torchvision

BATCH_SIZE = 4
WORKERS = 4
DROP = False

TRAIN_DATA_PATH = "./images/train/"
TEST_DATA_PATH = "./images/test/"
HOLDOUT_DATA_PATH = "./images/holdout/"

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=train_transforms)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=WORKERS, drop_last=DROP)

test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=test_transforms)
test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=WORKERS, drop_last=DROP)

holdout_data = torchvision.datasets.ImageFolder(root=HOLDOUT_DATA_PATH, transform=test_transforms)
holdout_data_loader  = data.DataLoader(holdout_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=WORKERS, drop_last=DROP)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class_names = train_data.classes
print(class_names)

#source: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_data_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
from torch import nn, optim
import pandas as pd
from timeit import default_timer as timer

model = torchvision.models.vgg19(pretrained=True)

In [ ]:
n_classes = 3

# Freeze early layers
for param in model.parameters():
    param.requires_grad = False
n_inputs = model.classifier[6].in_features

# Add on classifier
# modified from: https://towardsdatascience.com/transfer-learning-with-convolutional-neural-networks-in-pytorch-dd09190245ce
model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 256),
                      nn.ReLU(),
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),
                      nn.LogSoftmax(dim=1))

print(model.classifier)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=.001)

In [ ]:
# Move to gpu and parallelize
if train_on_gpu:
    model = model.to('cuda')

if multi_gpu:
    model = nn.DataParallel(model)

In [ ]:
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=1):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)

            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

In [ ]:
model, history = train(
    model,
    criterion,
    optimizer,
    train_data_loader,
    test_data_loader,
    save_file_name='dog_id_model',
    max_epochs_stop=20,
    n_epochs=45,
    print_every=2)

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_loss', 'valid_loss']:
    plt.plot(
        history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Negative Log Likelihood')
plt.title('Training and Validation Losses')

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_acc', 'valid_acc']:
    plt.plot(
        100 * history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Accuracy')
plt.title('Training and Validation Accuracy')


In [ ]:
holdout_loss = 0.0
holdout_acc = 0

#test_loader.dataset.t

with torch.no_grad():
    # Set to evaluation mode
    model.eval()

    # Validation loop
    for data, target in holdout_data_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        # Forward pass
        output = model(data)

        # Validation loss
        loss = criterion(output, target)
        # Multiply average loss times the number of examples in batch
        holdout_loss += loss.item() * data.size(0)

        # Calculate validation accuracy
        _, pred = torch.max(output, dim=1)
        correct_tensor = pred.eq(target.data.view_as(pred))
        accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
        # Multiply average accuracy times the number of examples
        holdout_acc += accuracy.item() * data.size(0)

    # Calculate average losses
    holdout_loss = holdout_loss / len(holdout_data_loader.dataset)

    # Calculate average accuracy
    holdout_acc = holdout_acc / len(holdout_data_loader.dataset)
    holdout_acc = round(holdout_acc*100, 2)

print(f'Holdout Accuracy: {holdout_acc}%')


# Extra Credit